# Analizza ricevute con Riconoscimento modulo

![Un robot che tiene in mano una ricevuta](./images/receipt_analysis.jpg)

Nel campo dell'intelligenza artificiale (IA) della visione artificiale, il riconoscimento ottico dei caratteri (OCR) è comunemente usato per leggere documenti stampati o scritti a mano. Spesso, il testo viene semplicemente estratto dai documenti in un formato che può essere utilizzato per ulteriori elaborazioni o analisi.

Uno scenario OCR più avanzato consiste nell'estrazione di informazioni da moduli, come ordini di acquisto o fatture, con una comprensione semantica di quanto rappresentato dai campi del modulo. Il servizio **Riconoscimento modulo** è specificamente progettato per questo tipo di problemi di AI.

## Visualizza una ricevuta

In questo esempio, userai il modello integrato di Riconoscimento modulo per analizzare le ricevute.

Fai clic sul pulsante **Esegui cella** (&#9655;) (a sinistra della cella) seguente per eseguirlo e vedere un esempio di ricevuta da analizzare usando Riconoscimento modulo.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline


# Load and display a receipt image
fig = plt.figure(figsize=(6, 6))
image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## Crea una risorsa Riconoscimento modulo

>**Nota:** È possibile utilizzare una risorsa di Servizio cognitivo o una risorsa di Riconoscimento modulo per accedere ai servizi di Riconoscimento modulo. 

Per creare una risorsa Riconoscimento modulo nella tua sottoscrizione di Azure:

1. In un'altra scheda del browser, apri il portale di Azure all'indirizzo https://portal.azure.com, accedendo con il tuo account Microsoft.
2. Seleziona **+ Crea una risorsa** e cerca *Riconoscimento modulo*.
3. Nell'elenco dei servizi, seleziona **Riconoscimento modulo**.
4. Nel pannello **Riconoscimento modulo**, seleziona **Crea**.
5. Nel pannello **Crea**, immetti i seguenti dettagli e seleziona **Crea**
   - **Nome**: Un nome univoco per il tuo servizio
   - **Sottoscrizione**: La tua sottoscrizione di Azure
   - **Area geografica**: Una qualsiasi area disponibile
   - **Piano tariffario**: F0
   - **Gruppo di risorse**: Il gruppo di risorse esistente che hai usato in precedenza
   - **Confermo di aver letto e compreso l'avviso seguente**: Selezionato.
6. Attendi che il servizio venga creato.
7. Visualizza il servizio Riconoscimento modulo appena creato nel portale di Azure e nella pagina **Chiavi ed endpoint**, copia i valori **Key1** ed **Endpoint** e incollali nella cella di codice sottostante, sostituendo **YOUR_FORM_KEY** e **YOUR_FORM_ENDPOINT**.

In [ ]:
form_key = 'YOUR_FORM_KEY'
form_endpoint = 'YOUR_FORM_ENDPOINT'

print('Ready to use form recognizer at {} using key {}'.format(form_endpoint, form_key))

## Analizza una ricevuta

Ora è tutto pronto per usare Riconoscimento modulo per analizzare una ricevuta.

In [ ]:
import os
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential

# Create a client for the form recognizer service
form_recognizer_client = FormRecognizerClient(endpoint=form_endpoint, credential=AzureKeyCredential(form_key))

try:
    print("Analyzing receipt...")
    # Get the receipt image file
    image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')

    # Submit the file data to form recognizer
    with open(image_path, "rb") as f:
        analyze_receipt = form_recognizer_client.begin_recognize_receipts(receipt=f)
    
    # Get the results
    receipt_data = analyze_receipt.result()

    # Print the extracted data for the first (and only) receipt
    receipt = receipt_data[0]
    receipt_type = receipt.fields.get("ReceiptType")
    if receipt_type:
        print("Receipt Type: {}".format(receipt_type.value))
    merchant_address = receipt.fields.get("MerchantAddress")
    if merchant_address:
        print("Merchant Address: {}".format(merchant_address.value))
    merchant_phone = receipt.fields.get("MerchantPhoneNumber")
    if merchant_phone:
        print("Merchant Phone: {}".format(merchant_phone.value))
    transaction_date = receipt.fields.get("TransactionDate")
    if transaction_date:
        print("Transaction Date: {}".format(transaction_date.value))
    print("Receipt items:")
    items = receipt.fields.get("Items")
    if items:
        for idx, item in enumerate(receipt.fields.get("Items").value):
            print("\tItem #{}".format(idx+1))
            item_name = item.value.get("Name")
            if item_name:
                print("\t - Name: {}".format(item_name.value))
            item_total_price = item.value.get("TotalPrice")
            if item_total_price:
                print("\t - Price: {}".format(item_total_price.value))
    subtotal = receipt.fields.get("Subtotal")
    if subtotal:
        print("Subtotal: {} ".format(subtotal.value))
    tax = receipt.fields.get("Tax")
    if tax:
        print("Tax: {}".format(tax.value))
    total = receipt.fields.get("Total")
    if total:
        print("Total: {}".format(total.value))

except Exception as ex:
    print('Error:', ex)


Tieni presente che Riconoscimento modulo è in grado di interpretare i dati nel modulo, identificando correttamente l'indirizzo e il numero di telefono del commerciante, la data e l'ora della transazione, così come le voci, la somma parziale, le tasse e gli importi totali.

## Ulteriori informazioni

Per ulteriori informazioni sul servizio Riconoscimento modulo, consulta la [documentazione di Riconoscimento modulo](https://docs.microsoft.com/it-it/azure/cognitive-services/form-recognizer/index)